# 申万行业数据SHAP分析

本Notebook用于分析指定日期前后短中期因子的权重变化。

## 1. 导入必要的库和模块

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 导入配置
from config import data_config, model_config, optimization_config, explanation_config, evaluation_config

# 导入数据处理模块
from data.data_generator import load_real_data

# 导入其他模块
from factors.momentum_factors import calculate_momentum_factors, get_factor_descriptions
from model.predictor import MarketTrendPredictor
from optimization.hyperparameter_tuning import optimize_hyperparameters, update_model_with_optimal_params
from explanation.model_explanation import create_shap_explainer, calculate_shap_values, analyze_factor_importance, detect_trend_changes
from utils.helpers import validate_data, format_metrics_for_display

import shap
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 2. 加载和处理数据

In [ ]:
# 加载申万行业数据
print('加载申万行业数据...')
raw_data = load_real_data('converted_sw_industry_data_v2.csv')
print(f'加载了 {len(raw_data)} 条数据记录')

# 计算动量因子
print('计算动量因子...')
factor_data = calculate_momentum_factors(raw_data, data_config)
print(f'计算了 {len(factor_data)} 条因子数据记录')

## 3. 训练模型

In [ ]:
# 创建预测器
predictor = MarketTrendPredictor(model_config)

# 准备特征和目标变量
features = predictor.prepare_features(factor_data)
target = predictor.prepare_target(factor_data)

# 验证数据
required_features = ['short_momentum', 'medium_momentum', 'long_momentum']
if not validate_data(features, required_features):
    print('特征数据验证失败')
else:
    print('特征数据验证通过')

# 超参数优化
print('进行超参数优化...')
optimization_result = optimize_hyperparameters(
    features, 
    target, 
    optimization_config
)
print(f"最优参数: {optimization_result['best_params']}")
print(f"最优值 (RMSE): {optimization_result['best_value']:.6f}")

# 使用最优参数更新模型
update_model_with_optimal_params(predictor, optimization_result['best_params'])

# 重新训练模型
predictor.train(features, target)
print('模型训练完成')

## 4. SHAP分析

In [ ]:
# 创建SHAP解释器
explainer = create_shap_explainer(predictor, features)

# 计算SHAP值
shap_values = calculate_shap_values(explainer, features, explanation_config)

# 分析因子重要性
importance_df = analyze_factor_importance(shap_values, features.columns.tolist())
print('因子重要性排序:')
print(importance_df.to_string(index=False))

## 5. 分析指定日期前后的因子权重变化

In [ ]:
import argparse

# 解析命令行参数
parser = argparse.ArgumentParser(description='分析指定日期前后短中期因子的权重变化')
parser.add_argument('--target_date', type=str, default='2025-04-07', help='目标日期 (格式: YYYY-MM-DD)')
args = parser.parse_args()

# 确定目标日期前后的数据
target_date = pd.Timestamp(args.target_date)

# 确保索引是日期类型
features_with_date = features.copy()
features_with_date['date'] = factor_data['date'].values
features_with_date['date'] = pd.to_datetime(features_with_date['date'])

# 分割数据
before_data = features_with_date[features_with_date['date'] < target_date]
after_data = features_with_date[features_with_date['date'] >= target_date]

print(f'目标日期: {target_date}')
print(f'目标日期前的数据点数量: {len(before_data)}')
print(f'目标日期后的数据点数量: {len(after_data)}')

# 计算两个时期的SHAP值
before_shap_values = calculate_shap_values(explainer, before_data.drop('date', axis=1), explanation_config)
after_shap_values = calculate_shap_values(explainer, after_data.drop('date', axis=1), explanation_config)

# 计算平均SHAP值
before_mean_shap = np.mean(np.abs(before_shap_values), axis=0)
after_mean_shap = np.mean(np.abs(after_shap_values), axis=0)

# 创建比较DataFrame
comparison_df = pd.DataFrame({
    'feature': features.columns.tolist(),
    'before_mean_shap': before_mean_shap,
    'after_mean_shap': after_mean_shap,
    'change': after_mean_shap - before_mean_shap,
    'change_percentage': (after_mean_shap - before_mean_shap) / before_mean_shap * 100
})

print('因子权重变化分析:')
print(comparison_df.to_string(index=False))

## 6. 可视化因子权重变化

In [ ]:
# 创建可视化
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(comparison_df['feature']))
width = 0.35

bars1 = ax.bar(x - width/2, comparison_df['before_mean_shap'], width, label='目标日期前', alpha=0.8)
bars2 = ax.bar(x + width/2, comparison_df['after_mean_shap'], width, label='目标日期后', alpha=0.8)

ax.set_xlabel('因子')
ax.set_ylabel('平均SHAP值')
ax.set_title(f'{args.target_date}前后因子权重变化')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['feature'])
ax.legend()

# 在柱状图上添加数值标签
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.5f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords='offset points',
                ha='center', va='bottom')

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.5f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords='offset points',
                ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 7. 结论

通过以上分析，我们可以观察到指定日期前后短中期因子的权重变化情况。这些变化可能反映了市场环境的转变，对投资决策具有重要参考价值。